### 数据处理

In [68]:
import numpy as np
import pandas as pd

In [69]:
data = pd.read_csv("C:/Users/huangm/Desktop/movie_comments.csv")

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
data.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [71]:
# 过滤缺失值
data.dropna(subset=['comment', 'star'], inplace=True)

In [72]:
# 
data.shape

(261495, 5)

In [73]:
# 查看star值的种类
data.star.value_counts()

4       43002
4       40806
3       33910
5       31947
3       31762
5       27368
2       14299
2       13837
1       12308
1       12255
star        1
Name: star, dtype: int64

In [74]:
# 替代star
data['star'].replace("star",1, inplace=True)
data['star'].replace("1", 1, inplace=True)
data['star'].replace("2",2, inplace=True)
data['star'].replace("3",3, inplace=True)
data['star'].replace("4",4, inplace=True)
data['star'].replace("5",5, inplace=True)
data.star.value_counts()

4    83808
3    65672
5    59315
2    28136
1    24564
Name: star, dtype: int64

In [75]:
data.star[:10]

0    1
1    2
2    2
3    4
4    1
5    1
6    2
7    4
8    4
9    1
Name: star, dtype: int64

In [76]:
# X = data.drop(['id','link','name', 'star'], axis=1)
# y = data.drop(['id','link','name','comment'], axis=1)
# X.head()

In [77]:
comments = data.comment.tolist()
stars = data.star.tolist()
len(comments), type(comments), len(stars), type(stars)

(261495, list, 261495, list)

In [78]:
import jieba
import re

def cut(string):
    """切词"""
    return ' '.join(jieba.cut(string))

def token(string):
    """匹配文字"""
    return re.findall(r'[\d\w]+', string)

# with open('../../data/comments.txt', 'w', encoding="utf-8") as f:
#     for comment in comments:
#         contents = cut(''.join(token(comment)))
#         f.write(contents + '\n')

In [79]:
comments = [cut(''.join(token(comment))) for comment in comments]
stars = [int(star) for star in stars]

### 向量化

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer # sklearn tfidf向量化

vectorized = TfidfVectorizer(ngram_range=(1, 3), max_features=200)
comments_vectors = vectorized.fit_transform(comments)
comments_vectors.shape

(261495, 200)

In [81]:
comments_array = comments_vectors.toarray()

### 训练集和数据集

In [82]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(comments_array, stars, test_size=0.2, random_state=52)
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=32)
x_train.shape, len(y_train), x_valid.shape, len(y_valid), x_test.shape, len(y_test)

((167356, 200), 167356, (41840, 200), 41840, (52299, 200), 52299)

### 构建神经网络

In [83]:
def reformat(labels):
    """Reformat label style"""
    labels = (np.arange(1, 6) == labels[:,None]).astype(np.float32) 
    return labels

y_train = reformat(np.array(y_train))
y_valid = reformat(np.array(y_valid))
y_test = reformat(np.array(y_test))
print(y_train.shape, y_valid.shape, y_test.shape)
x_train = x_train.astype(np.float32)
x_valid = x_valid.astype(np.float32) 
x_test = x_test.astype(np.float32) 

(167356, 5) (41840, 5) (52299, 5)


In [84]:
def accuracy(y_pred, y_true):
    """compute accuracy"""
    return 100 * np.sum(np.argmax(y_pred, 1) == np.argmax(y_true, 1)) / y_pred.shape[0]

In [85]:
import math
import tensorflow as tf

hidden1_units = 500 # Number of first hidden layer units
hidden2_units = 400 # Number of second hidden layer units

num_classes = 5
batch_size = 10000    
num_epochs = 20

initial_learning_rate = 0.5
learning_rate_decay_factor = 0.95
num_epochs_per_decay = 100

lam = 0.002

# Dimension of train sets
d = x_train.shape
n_train_samples = d[0] # the num of train sets
n_features = d[1]  # features num

# compute graph
graph = tf.Graph()
with graph.as_default():
    # Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, n_features), name="comment")
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_classes), name='star')
    tf_valid_dataset = tf.constant(x_valid)
    tf_test_dataset = tf.constant(x_test)
    
    # 1 layer
    weights1 = tf.Variable(
        tf.truncated_normal([n_features, hidden1_units],
                           stddev=1.0 / math.sqrt(float(n_features))), name="weights1")
    biases1 = tf.Variable(tf.zeros([hidden1_units]), name="biases1")
    
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    
    # 2 layer
    weights2 = tf.Variable(
        tf.truncated_normal([hidden1_units, hidden2_units],
                           stddev=1.0 / math.sqrt(float(hidden1_units))), name="weights2")
    biases2 = tf.Variable(tf.zeros([hidden2_units]), name="biases2")
    
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
    
    # 3 layer
    weights3 = tf.Variable(
        tf.truncated_normal([hidden2_units, num_classes],
                           stddev=1.0 / math.sqrt(float(hidden2_units))), name="weights3")
    biases3 = tf.Variable(tf.zeros([num_classes]), name="biases3")
    
    # 增加dropout, 改善过拟合
    keep_pro = tf.placeholder(tf.float32)
    hidden2_drop = tf.nn.dropout(hidden2, keep_pro)
    
    # Train computation
    logits = tf.matmul(hidden2_drop, weights3) + biases3
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
        lam * tf.nn.l2_loss(weights1) +
        lam * tf.nn.l2_loss(weights2) +
        lam * tf.nn.l2_loss(weights3)) # 增加 正则项
     
    # global step
    global_step = tf.Variable(0, name="global_step", trainable=False)

    # learning rate policy
    decay_steps = int(n_train_samples / batch_size * num_epochs_per_decay)
    learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                               global_step,
                                               decay_steps,
                                               learning_rate_decay_factor,
                                               staircase=True,
                                               name="exponential_decay_learning_rate")
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training
    train_prediction = tf.nn.softmax(logits)
    
    # validation
    hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    hidden_valid_2 = tf.nn.relu(tf.matmul(hidden_valid_1, weights2) + biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid_2, weights3) + biases3)
    
    # test
    hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1, weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(hidden_test_2, weights3) + biases3)

In [86]:
with tf.Session(graph=graph) as sess:
    # 初始化
    sess.run(tf.global_variables_initializer())
    print("Initialized")
    
    for epoch in range(num_epochs):
        total_batch_training = int(n_train_samples / batch_size) # train 次数 per epoch

        for batch_num in range(total_batch_training):
            start = batch_num * batch_size
            end = (batch_num + 1) * batch_size

            # Fit training using batch data
            train_batch_data, train_batch_labels = x_train[start:end], y_train[start:end]
            
            # run session
            batch_loss, _, predictions = sess.run(
                [loss, optimizer, train_prediction],
            feed_dict={tf_train_dataset:train_batch_data, tf_train_labels:train_batch_labels, keep_pro:0.5})
            
        if epoch % 5 == 0:
            print("Loss at step %d: %f" % (epoch, batch_loss))
            print("Train accuracy: %.1f%%" % accuracy(predictions, train_batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval({tf_train_dataset: train_batch_data, tf_train_labels:train_batch_labels, keep_pro:1}), y_valid)) # 1
            print("Test accuracy: %.1f%%" % accuracy(
                test_prediction.eval({tf_train_dataset: train_batch_data, tf_train_labels:train_batch_labels, keep_pro:1}), y_test)) # 1

Initialized
Loss at step 0: 2.191806
Train accuracy: 32.0%
Validation accuracy: 32.3%
Test accuracy: 32.3%
Loss at step 5: 2.085783
Train accuracy: 32.0%
Validation accuracy: 32.3%
Test accuracy: 32.3%
Loss at step 10: 1.994474
Train accuracy: 32.1%
Validation accuracy: 32.3%
Test accuracy: 32.3%
Loss at step 15: 1.909983
Train accuracy: 32.9%
Validation accuracy: 33.0%
Test accuracy: 33.0%


### Fasttext

In [97]:
data.head()

,comment,star
0,吴京意淫到了脑残的地步，看了恶心想吐,1
1,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,中二得很,1


In [103]:
# data = data.drop(['id','link','name'], axis=1)
data.shape

(261495, 2)

In [104]:
train_data, test_data = train_test_split(data, random_state=20)
train_data.shape, test_data.shape

((196121, 2), (65374, 2))

In [109]:
import jieba
import re
import random
import fasttext

def cut(string):
    """切词"""
    return list(jieba.cut(string))

def token(string):
    """匹配文字"""
    return re.findall(r'[\d\w]+', string)

def get_stopwords(filename):
    """停用词"""
    stopwords = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            stopwords.append(line.strip())
    return set(stopwords)
        
def preprocess_text(comments, stopwords):
    """预处理"""
    comment_list = []
    for comment, star in comments.values:
        words = cut(''.join(token(comment))) # 分词 and 正则
        words = filter(lambda x:len(x)>1, words)
        words = filter(lambda x:x not in stopwords, words)
        comment_list.append('__label__' + str(star) + ' ' + ''.join(words))
    return comment_list

def save_to_file(comment_list, filename):
    with open(filename, 'w', encoding="utf-8") as f:
        for comment in comment_list:
            f.write(comment + '\n')

def preprocess_data(comments, stopwords, save_filename):
    
    # 预处理
    comment_list = preprocess_text(comments, stopwords)
    
    # 训练数据
    save_to_file(comment_list, save_filename)

In [139]:
stopwords_file_path = './chinese_stopwords.txt'
save_file_path_1 = '../../data/train_data.txt'
save_file_path_2 = '../../data/test_data.txt'

stopwords = get_stopwords(stopwords_file_path)

preprocess_data(train_data, stopwords, save_file_path_1)
preprocess_data(test_data, stopwords, save_file_path_2)

# Fasttext监督学习
ft = fasttext.train_supervised(save_file_path_1, lr=1e-2, epoch=20,  wordNgrams=2)

ft.test(save_file_path_1),ft.test(save_file_path_2)

((196121, 0.945115515421602, 0.945115515421602),
 (65374, 0.409566494324961, 0.409566494324961))

In [140]:
ft.predict('放假了算法思路放好了')[0][0][-1]

'3'

In [141]:
test_data.values[:5]

array([['4星半', 5],
       ['我个人非常不喜欢这片子，也许是不喜欢近几年这类型的所有片子：一股阴湿腻味的腥气扑面而来。里头许多角色貌似都是得过精神病的甲亢患者，跟孙海英入过一个教（真有个教主唉）。有些武侠有侠气，有些有鬼气，有些是仙气，而有些只剩下血腥气和肉膈气。纯靠真爱的面孔和川普撑到最后。',
        2],
       ['后半段没有共鸣', 4],
       ['导演是不是也想某天被意外撞死啊？？？？？？', 3],
       ['歌好听', 4]], dtype=object)

In [142]:
train_pred_list = []
for x, _ in train_data.values:
    x = ''.join(cut(''.join(token(x))))
    y_pred = ft.predict(x)[0][0][-1]
    train_pred_list.append(int(y_pred))
train_pred = np.array(train_pred_list)
train_pred.shape

(196121,)

In [143]:
y_true= train_data['star'].values
y_true.shape

(196121,)

In [144]:
(train_pred == y_true).mean()

0.32184722696702545

In [145]:
test_pred_list = []
for x, _ in test_data.values:
    x = ''.join(cut(''.join(token(x))))
    y_pred = ft.predict(x)[0][0][-1]
    test_pred_list.append(int(y_pred))
test_pred = np.array(test_pred_list)
test_pred.shape

(65374,)

In [146]:
y_true= test_data['star'].values
y_true.shape

(65374,)

In [147]:
(test_pred == y_true).mean()

0.26951081469697435